In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
walks = pd.read_csv("data/walks.csv")
walkers = pd.read_csv("data/walkers.csv")
demands = pd.read_csv("data/demands.csv")

In [3]:
walks[:2]

,Unnamed: 0,ownerid,walkid,walkerid,walkingid,dogid,walkingtype,duration,checkintime,distance,district,neighborhood,endtime
0,0,eefa1cdf-041b-4738-a0f5-6e6647b28b1a,fdb4fdd3-19b7-4176-a2ee-817eeaba4f4d,09e59ced-1172-4744-a2bb-eb68ae83041c,e5b70802-a457-472a-9852-cba0889ef85a,1e68f0a4-101d-44fb-b9d6-add3277f1277,Customize,45,2019-12-06 07:00:00.000,2.826252,Beşiktaş,NaN,2019-12-06 08:06:06.853
1,1,b8ed3dd3-8a62-4189-85ee-b224913c64d6,e02af026-f364-40c4-9483-0adb4e449ed2,6007a0a8-d40b-4560-9ead-180c1646dae5,0d1f6547-7f63-4231-ace7-b23692102b70,fe49e24b-87ee-4512-b9e5-880e61021617,Customize,45,2020-03-03 07:30:00.000,1.241175,Kadıköy,NaN,2020-03-03 08:22:12.239


In [4]:
walkers[:2]

,Unnamed: 0,id,gender,signuptime,walker_province,walker_district,walker_neighborhood
0,0,248b0c40-6670-47e1-be2d-052934fac3ea,Female,2019-11-03 22:01:38.006,İstanbul,Ataşehir,NaN
1,1,01029542-74c8-4ba7-9370-c63d0eab244d,Male,2020-02-11 15:30:28.371,İstanbul,Maltepe,NaN


In [5]:
demands[:2]

,orderid,walkerid,demandtime,dogid,finishedservicecount,finishedservicecountbydogid,ispreferred,matchingid,ownerid,servicetype,walkerdistrict,walkerneighborhood,walkerdograteresponse,walkergeo,walkergroup,walkerrateresponse
0,e44b27ec-2acb-4254-a9b4-b24bd0da7d8c,29b62d7f-61bf-4622-bd2c-af5044a398b5,2020-08-25 16:22:25.397,9dc9b910-4d9e-4d8d-acc1-2c7b36974cfa,78,2,False,31b72223-3855-49fc-a958-8e955e1c61d4,43b72967-b365-4d81-a283-4f77c5662f68,AdHoc,Kadıköy,Caferağa,"(0.0, 0)","(40.98659, 29.02385)",TypeC,"(5.0, 1)"
1,bbfece32-244a-4ed0-acf2-51c7ec3cb347,e8220c30-3470-44af-b7e3-3f2a063e6ecf,2020-07-29 19:57:53.518,9fa36b2d-2097-4bdd-9fe6-8742d93fbbb3,1,0,NaN,a5d2ba0c-f41f-411b-a4be-c466a99ad182,ea27cea2-0876-41bc-b8e8-5af5402b34a7,Planned,Maltepe,Çınar,"(0.0, 0)","(40.94159, 29.11359)",TypeB,"(0.0, 0)"


In [6]:
new = pd.DataFrame()
new["walkerId"] = pd.DataFrame(walkers["id"])
new[:2]

,walkerId
0,248b0c40-6670-47e1-be2d-052934fac3ea
1,01029542-74c8-4ba7-9370-c63d0eab244d


In [7]:
total_walks = pd.DataFrame()
total_walks = pd.DataFrame(walks["walkerid"].value_counts()).reset_index()
total_walks = total_walks.rename(columns={"index": "walkerId", "walkerid": "total_walks"})

total_walks[:2]

,walkerId,total_walks
0,7594081d-eeef-41d3-b26e-c92e73c11863,595
1,5b9a6efe-4622-4cb0-a330-bb8494e81a77,532


In [8]:
new = pd.merge(new, total_walks, on='walkerId', how='left')
new[:2]

,walkerId,total_walks
0,248b0c40-6670-47e1-be2d-052934fac3ea,NaN
1,01029542-74c8-4ba7-9370-c63d0eab244d,NaN


In the check below, we see that a person who is not registered in walkers.csv can order a walk. This is most likely due to walkers.csv being truncated. I will continue ignoring this factor.

In [9]:
print(new[new["walkerId"] == '6007a0a8-d40b-4560-9ead-180c1646dae5'])
print(total_walks[total_walks["walkerId"] == '6007a0a8-d40b-4560-9ead-180c1646dae5'])

Empty DataFrame
Columns: [walkerId, total_walks]
Index: []
                               walkerId  total_walks
6  6007a0a8-d40b-4560-9ead-180c1646dae5          274


In [10]:
ordered = pd.DataFrame()
ordered["ownerId"] = walks["ownerid"].unique()

oldest_dates = {}
newest_dates = {}

for index, row in walks.iterrows():
    owner_id = row['ownerid']
    demand_time = row['endtime']
    
    if (owner_id not in oldest_dates) or (demand_time < oldest_dates[owner_id]):
        oldest_dates[owner_id] = demand_time
        
    if (owner_id not in newest_dates) or (demand_time > newest_dates[owner_id]):
        newest_dates[owner_id] = demand_time

firstOrder_df = pd.DataFrame({'ownerId': list(oldest_dates.keys()), 'firstDemand': list(oldest_dates.values())})
lastOrder_df = pd.DataFrame({'ownerId': list(newest_dates.keys()), 'lastDemand': list(newest_dates.values())})

ordered = pd.merge(ordered, firstOrder_df, on='ownerId', how='left')
ordered = pd.merge(ordered, lastOrder_df, on='ownerId', how='left')


ordered[:2]

,ownerId,firstDemand,lastDemand
0,eefa1cdf-041b-4738-a0f5-6e6647b28b1a,2019-10-17 08:03:11.373,2021-02-18 14:26:40.217
1,b8ed3dd3-8a62-4189-85ee-b224913c64d6,2019-01-08 07:18:13.018,2020-09-27 16:15:34.661


In [11]:
oldest_dates = {}
newest_dates = {}

for index, row in walks.iterrows():
    walker_id = row['walkerid']
    demand_time = row['checkintime']
    
    if (walker_id not in oldest_dates) or (demand_time < oldest_dates[walker_id]):
        oldest_dates[walker_id] = demand_time
        
    if (walker_id not in newest_dates) or (demand_time > newest_dates[walker_id]):
        newest_dates[walker_id] = demand_time

firstWalk_df = pd.DataFrame({'walkerId': list(oldest_dates.keys()), 'firstWalk': list(oldest_dates.values())})
lastWalk_df = pd.DataFrame({'walkerId': list(newest_dates.keys()), 'lastWalk': list(newest_dates.values())})

new = pd.merge(new, firstWalk_df, on='walkerId', how='left')
new = pd.merge(new, lastWalk_df, on='walkerId', how='left')

In [12]:
new = pd.merge(new, walkers[['id', 'signuptime']], left_on='walkerId', right_on='id', how='left')
new.drop(columns=['id'], inplace=True)

In [13]:
new = new.rename(columns={"signuptime": "signupTime"})

In [14]:
new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648


In [15]:
isLive = dict()

for index,row in walks.iterrows():
    walker = row["walkerid"]
    if not walker in isLive:
        isLive[walker] = "Yes"

for index,row in demands.iterrows():
    walker = row["walkerid"]
    if not walker in isLive:
        isLive[walker] = "Yes"

new["didDemandOrWalk"] = new["walkerId"].map(isLive).fillna("No")
new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes


In [16]:
lifetime = list()
total_litetime = 0

for index,row in new.iterrows():
    row["signupTime"] = pd.to_datetime(row["signupTime"])
    row["lastWalk"] = pd.to_datetime(row["lastWalk"])
    signupTime = row["signupTime"]
    lastWalk = row["lastWalk"]

    lt = lastWalk - signupTime
    lifetime.append(lt.days)
    if (pd.isnull(signupTime) or pd.isnull(lastWalk)):
        continue
    else:
        total_litetime += lt.days

new["lifetime"] = lifetime

a_lifetime = total_litetime / new.shape[0]
average_lifetime = list()
for day in new["lifetime"]:
    if math.isnan(day):
        average_lifetime.append("NaN")
    elif(day < a_lifetime):
        average_lifetime.append("Less")
    else:
        average_lifetime.append("More")

new["averageLifetime"] = average_lifetime

new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less


In [17]:
total_walks = dict()
total_km = 0
total_walker = new.shape[0]

for index,row in walks.iterrows():
    id = row["walkerid"]
    km = row["distance"]

    if id in total_walks:
        total_walks[id] += km
    else:
        total_walks[id] = km
    
    total_km += km

a_km = total_km / total_walker

average_walks = dict()
for key,value in total_walks.items():
    if(total_walks[key] < a_km):
        average_walks[key] = "Less"
    else:
        average_walks[key] = "More"

new["averageWalk"] = new["walkerId"].map(average_walks)   

In [18]:
firstwalk_aftersignup = list()

for index,row in new.iterrows():
    signup = pd.to_datetime(row["signupTime"])
    firstw = pd.to_datetime(row["firstWalk"])

    diff = (firstw - signup).days
    firstwalk_aftersignup.append(diff)

new["firstWalkAfterSignup"] = firstwalk_aftersignup
new[3:5]   

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0


In [19]:
fw_inWeek = list()
for index,row in new.iterrows():
    fw = row["firstWalkAfterSignup"]
    if math.isnan(day):
        fw_inWeek.append("No")
    elif(fw <= 7):
        fw_inWeek.append("Yes")
    else:
        fw_inWeek.append("No")

new["firstWalkinWeek"] = fw_inWeek
new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No


In [20]:
last_time_in_program = pd.to_datetime("2019-11-03 22:01:38.006")
for index,row in walks.iterrows():
    last = pd.to_datetime(row["checkintime"])

    if(last > last_time_in_program):
        last_time_in_program = last
        
print(last_time_in_program)
temp = last_time_in_program

# Task Wanted
last_time_in_program = pd.to_datetime("today").normalize()
print(last_time_in_program)

# I prefer to use, for more logical outputs
last_time_in_program = temp

2021-02-26 17:45:00
2024-05-01 00:00:00


In [21]:
last_3day_walks = dict()
for index,row in walks.iterrows():
    id = row["walkerid"]
    walk_end = pd.to_datetime(row["endtime"])

    day_diff = (last_time_in_program - walk_end).days

    if id in last_3day_walks:
        if(day_diff <= 30):
            last_3day_walks[id] += 1
    else:
        last_3day_walks[id] = 0

new["lastMonthWalks"] = new["walkerId"].map(last_3day_walks).fillna(0)
new[3:5]


,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0


In [22]:
isActive = dict()

for index,row in walks.iterrows():
    time = pd.to_datetime(row["endtime"])

    owner = row["ownerid"]
    walker = row["walkerid"]

    diff_day = (last_time_in_program - time).days

    if(diff_day <= 30):
        isActive[owner] = "Active"
        isActive[walker] = "Active"
    
new["activityTag"] = new["walkerId"].map(isActive).fillna("NotActive")
new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks,activityTag
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0,NotActive
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0,NotActive


In [23]:
district_counts_per_walker = pd.DataFrame()
district_counts_per_walker = walks.groupby("walkerid")["district"].nunique().reset_index()
district_counts_per_walker.columns = ["walkerId", "differentDistrictCount"]

new = pd.merge(new, district_counts_per_walker, on='walkerId', how='left')

new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks,activityTag,differentDistrictCount
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0,NotActive,1.0
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0,NotActive,1.0


In [24]:
first_and_last = dict()

for index,row in ordered.iterrows():
    id = row["ownerId"]
    first = pd.to_datetime(row["firstDemand"])
    last = pd.to_datetime(row["lastDemand"])

    if not id in first_and_last:
        first_and_last[id] = [first, last]
    else:
        if(first_and_last[id][0] > first):
            first_and_last[id][0] = first
        if(first_and_last[id][1] < last):
            first_and_last[id][1] = last

for index,row in new.iterrows():
    id = row["walkerId"]
    first = pd.to_datetime(row["firstWalk"])
    last = pd.to_datetime(row["lastWalk"])

    if not id in first_and_last:
        first_and_last[id] = [first, last]
    else:
        if(first_and_last[id][0] > first):
            first_and_last[id][0] = first
        if(first_and_last[id][1] < last):
            first_and_last[id][1] = last


activeLifeTime = dict()

for key,value in first_and_last.items():
    first = value[0]
    last = value[1]
    diff_day = (last - first)

    activeLifeTime[key] = diff_day

new["activeLifeTime"] = new["walkerId"].map(activeLifeTime).fillna("NaT")

new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks,activityTag,differentDistrictCount,activeLifeTime
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0,NotActive,1.0,90 days 01:34:21.821000
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0,NotActive,1.0,0 days 00:00:00


In [25]:
totalActiveDayCount = dict()

for index,row in walks.iterrows():
    id = row["walkerid"]
    check = pd.to_datetime(row["checkintime"])
    ended = pd.to_datetime(row["endtime"])

    duration = ended - check
    if id in totalActiveDayCount:
        totalActiveDayCount[id] += duration
    else:
        totalActiveDayCount[id] = duration

new["totalActiveDayCount"] = new["walkerId"].map(totalActiveDayCount).fillna(pd.Timedelta(0))
new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks,activityTag,differentDistrictCount,activeLifeTime,totalActiveDayCount
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0,NotActive,1.0,90 days 01:34:21.821000,0 days 08:54:24.522000
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0,NotActive,1.0,0 days 00:00:00,0 days 00:43:45.778000


In [26]:
walkFrequency = dict()
for index,row in new.iterrows():
    id = row["walkerId"]
    totalActiveDayCount = pd.to_timedelta(row["totalActiveDayCount"]).total_seconds() / (24 * 3600)
    activeLifeTime = pd.to_timedelta(row["activeLifeTime"]).total_seconds() / (24 * 3600)


    if(totalActiveDayCount == 0):
        walkFrequency[id] = 0
    else:
        walkFrequency[id] = activeLifeTime/totalActiveDayCount

new["walkFrequency"] = new["walkerId"].map(walkFrequency).fillna(0)

new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks,activityTag,differentDistrictCount,activeLifeTime,totalActiveDayCount,walkFrequency
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0,NotActive,1.0,90 days 01:34:21.821000,0 days 08:54:24.522000,242.687598
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0,NotActive,1.0,0 days 00:00:00,0 days 00:43:45.778000,0.000000


In [27]:
dailyWalkAverage = {}

for index, row in new.iterrows():
    id = row["walkerId"]
    totalWalk = row["total_walks"]
    totalActiveDayCount = pd.to_timedelta(row["totalActiveDayCount"]).total_seconds() / (24 * 3600)

    if(totalActiveDayCount == 0):
        dailyWalkAverage[id] = 0
    else:
        dailyWalkAverage[id] = float(totalWalk) / float(totalActiveDayCount)

new["dailyWalkAverage"] = new["walkerId"].map(dailyWalkAverage)

new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks,activityTag,differentDistrictCount,activeLifeTime,totalActiveDayCount,walkFrequency,dailyWalkAverage
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0,NotActive,1.0,90 days 01:34:21.821000,0 days 08:54:24.522000,242.687598,32.334803
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0,NotActive,1.0,0 days 00:00:00,0 days 00:43:45.778000,0.000000,32.904533


In [28]:
totalDemandCount = dict()

for index,row in demands.iterrows():
    walkerId = row["walkerid"]

    if walkerId in totalDemandCount:
        totalDemandCount[walkerId] += 1
    else:
        totalDemandCount[walkerId] = 1

new["totalDemandCount"] = new["walkerId"].map(totalDemandCount).fillna(0)

In [29]:
last30DaysDemandCount = {}

for index, row in demands.iterrows():
    id = row["walkerid"]
    demandtime = pd.to_datetime(row["demandtime"])
    day_diff = (last_time_in_program - demandtime).days

    if id in last30DaysDemandCount:
        if day_diff <= 30:
            last30DaysDemandCount[id] += 1
    else:
        last30DaysDemandCount[id] = 0

new["last30DaysDemandCount"] = new["walkerId"].map(last30DaysDemandCount).fillna(0)
new[3:5]


,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,firstWalkinWeek,lastMonthWalks,activityTag,differentDistrictCount,activeLifeTime,totalActiveDayCount,walkFrequency,dailyWalkAverage,totalDemandCount,last30DaysDemandCount
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,No,0.0,NotActive,1.0,90 days 01:34:21.821000,0 days 08:54:24.522000,242.687598,32.334803,62.0,2.0
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,No,0.0,NotActive,1.0,0 days 00:00:00,0 days 00:43:45.778000,0.000000,32.904533,5.0,0.0


In [30]:
weeklyAverageDemandCount = dict()

for index, row in new.iterrows():
    walkerId = row["walkerId"]
    totalDemandCount = row["totalDemandCount"]
    activeLifeTime = row["activeLifeTime"]
    
    if pd.isnull(activeLifeTime):
        weeklyAverageDemandCount[walkerId] = "NaN"
    else:
        activeLifeTime_days = activeLifeTime.total_seconds() / (24 * 3600)  
        
        if activeLifeTime_days == 0:
            weeklyAverageDemandCount[walkerId] = "NaN"
        else:
            weeklyAverageDemandCount[walkerId] = totalDemandCount / (activeLifeTime_days / 7) 


new["weeklyAverageDemandCount"] = new["walkerId"].map(weeklyAverageDemandCount).fillna("NaN")

new[3:5]

,walkerId,total_walks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,...,lastMonthWalks,activityTag,differentDistrictCount,activeLifeTime,totalActiveDayCount,walkFrequency,dailyWalkAverage,totalDemandCount,last30DaysDemandCount,weeklyAverageDemandCount
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,...,0.0,NotActive,1.0,90 days 01:34:21.821000,0 days 08:54:24.522000,242.687598,32.334803,62.0,2.0,4.818714
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,...,0.0,NotActive,1.0,0 days 00:00:00,0 days 00:43:45.778000,0.000000,32.904533,5.0,0.0,NaN


In [31]:
new.rename(columns={"total_walks":"totalWalks"}, inplace=True)

In [32]:
new.to_csv("output/Walker_Basic_Info_Table.csv", index=False)